In [4]:
#Importing PySpark and SparkContext
import pyspark
from pyspark import SparkContext
sc =SparkContext()

In [4]:
#Reading data into a rdd
rdd = sc.textFile('AmazonReviewData.csv')

In [5]:
#Displaying Headers
rdd.first()

'id,Name,asins,brand,categories,keys,manufacturer,ReviewsDate,ReviewsdateAdded,ReviewsDateSeen,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username'

In [6]:
#Displaying First line of data
rdd_head = rdd.first()
rdd1 = rdd.filter(lambda line:line != rdd_head)
rdd1.first()

'AVqkIhwDv8e3D1O-lebb,All-New Fire HD 8 Tablet- 8 HD,B01AHB9CN2,Amazon,Electronics-iPad  - Tablets-All Tablets-Fire Tablets-Tablets-Computers  - Tablets,841667104676-amazon/53004484-amazon/b01ahb9cn2-0841667104676-allnewfirehd8tablet8hddisplaywifi16gbincludesspecialoffersmagenta/5620406-allnewfirehd8tablet8hddisplaywifi16gbincludesspecialoffersmagenta/b01ahb9cn2,Amazon,1/13/17,7/3/17,6/7/17, FALSE,TRUE,,0,5,This product so far has not disappointed. My children love to use it and I like the ability to monitor control what content they see with ease.,Kindle,,,Adapter'

In [7]:
rdd1.map(lambda line:line.split(',')).take(1)

[['AVqkIhwDv8e3D1O-lebb',
  'All-New Fire HD 8 Tablet- 8 HD',
  'B01AHB9CN2',
  'Amazon',
  'Electronics-iPad  - Tablets-All Tablets-Fire Tablets-Tablets-Computers  - Tablets',
  '841667104676-amazon/53004484-amazon/b01ahb9cn2-0841667104676-allnewfirehd8tablet8hddisplaywifi16gbincludesspecialoffersmagenta/5620406-allnewfirehd8tablet8hddisplaywifi16gbincludesspecialoffersmagenta/b01ahb9cn2',
  'Amazon',
  '1/13/17',
  '7/3/17',
  '6/7/17',
  ' FALSE',
  'TRUE',
  '',
  '0',
  '5',
  'This product so far has not disappointed. My children love to use it and I like the ability to monitor control what content they see with ease.',
  'Kindle',
  '',
  '',
  'Adapter']]

In [5]:
from pyspark.sql import*

In [6]:
sqlContext = SQLContext(sc)

In [7]:
df=sqlContext.read.csv("AmazonReviewData.csv", header=True)

In [11]:
df.count()

34658

In [12]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- asins: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- keys: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- ReviewsDate: string (nullable = true)
 |-- ReviewsdateAdded: string (nullable = true)
 |-- ReviewsDateSeen: string (nullable = true)
 |-- reviews.didPurchase: string (nullable = true)
 |-- reviews.doRecommend: string (nullable = true)
 |-- reviews.id: string (nullable = true)
 |-- reviews.numHelpful: string (nullable = true)
 |-- reviews.rating: string (nullable = true)
 |-- reviews.text: string (nullable = true)
 |-- reviews.title: string (nullable = true)
 |-- reviews.userCity: string (nullable = true)
 |-- reviews.userProvince: string (nullable = true)
 |-- reviews.username: string (nullable = true)



In [8]:
df = df.withColumnRenamed("id","ProductID")
df = df.withColumnRenamed("name","ProductName")
df = df.withColumnRenamed("asins","ProductAsins")
df = df.withColumnRenamed("brand","ProductBrand")
df = df.withColumnRenamed("categories","ProductCategory")
df = df.withColumnRenamed("key","ProductKey")
df = df.withColumnRenamed("manufacturer","ProductManufacturer")
df = df.withColumnRenamed("ReviewsDate","ProductLauncheddate")
df = df.withColumnRenamed("reviews.doRecommended","RecommendStatus")
df = df.withColumnRenamed("reviews.id","ReviewID")
df = df.withColumnRenamed("reviews.numHelpful","ReviewHelpfulStatus")
df = df.withColumnRenamed("reviews.rating","ReviewRating")
df = df.withColumnRenamed("reviews.sourceURLs","ReviewURL")
df = df.withColumnRenamed("reviews.text","ReviewText")
df = df.withColumnRenamed("reviews.title","ReviewTitle")
df = df.withColumnRenamed("reviews.username","ReviewUserName")


In [18]:
df.printSchema()

root
 |-- ProductID: string (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- ProductAsins: string (nullable = true)
 |-- ProductBrand: string (nullable = true)
 |-- ProductCategory: string (nullable = true)
 |-- keys: string (nullable = true)
 |-- ProductManufacturer: string (nullable = true)
 |-- ProductLauncheddate: string (nullable = true)
 |-- ReviewsdateAdded: string (nullable = true)
 |-- ReviewsDateSeen: string (nullable = true)
 |-- PurchaseStatus: string (nullable = true)
 |-- RecommendStatus: string (nullable = true)
 |-- ReviewID: string (nullable = true)
 |-- ReviewHelpfulStatus: string (nullable = true)
 |-- ReviewRating: string (nullable = true)
 |-- ReviewText: string (nullable = true)
 |-- ReviewTitle: string (nullable = true)
 |-- ReviewUserCity: string (nullable = true)
 |-- ReviewUserProvince: string (nullable = true)
 |-- ReviewUserName: string (nullable = true)



In [9]:
df = df.withColumnRenamed("reviews.userCity","ReviewUserCity")
df = df.withColumnRenamed("reviews.userProvince","ReviewUserProvince")

In [10]:
df = df.withColumnRenamed("reviews.didPurchase","PurchaseStatus")
df = df.withColumnRenamed("reviews.doRecommend","RecommendStatus")

In [21]:
from pyspark.sql.functions import col,isnan,when,count
df_Columns=["ReviewsdateAdded","ReviewsDateSeen","ProductLaunchedDate"]
df2 = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df_Columns])
df2.show()

+----------------+---------------+-------------------+
|ReviewsdateAdded|ReviewsDateSeen|ProductLaunchedDate|
+----------------+---------------+-------------------+
|           10619|              0|                 39|
+----------------+---------------+-------------------+



In [22]:
from pyspark.sql.functions import coalesce
    
df = df.withColumn("ReviewsdateAdded",coalesce(df.ReviewsdateAdded,df.ReviewsDateSeen))

In [27]:
df = df.withColumn("ProductLauncheddate",coalesce(df.ProductLauncheddate,df.ReviewsdateAdded))

In [28]:
from pyspark.sql.functions import col,isnan,when,count
df_Columns=["ReviewsdateAdded","ReviewsDateSeen","ProductLaunchedDate"]
df2 = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df_Columns])
df2.show()

+----------------+---------------+-------------------+
|ReviewsdateAdded|ReviewsDateSeen|ProductLaunchedDate|
+----------------+---------------+-------------------+
|               0|              0|                  0|
+----------------+---------------+-------------------+



In [11]:
df.registerTempTable('table')

In [12]:
sqlContext.sql('select distinct RecommendStatus from table').show(10)

+---------------+
|RecommendStatus|
+---------------+
|          FALSE|
|           null|
|           TRUE|
+---------------+



In [13]:
sqlContext.sql('select distinct PurchaseStatus from table').show(10)

+--------------+
|PurchaseStatus|
+--------------+
|          TRUE|
|         FALSE|
+--------------+



In [37]:
from pyspark.sql.functions import when
df = df.withColumn("RecommendStatus", when(df.RecommendStatus == True, 1).otherwise(0))

In [49]:
df = df.withColumn("PurchaseStatus", when(df.PurchaseStatus == True, 1).otherwise(0))

In [48]:
sqlContext.sql('select distinct RecommendStatus from table where RecommendStatus = 1').show(10)

+---------------+
|RecommendStatus|
+---------------+
+---------------+



In [50]:
sqlContext.sql('select distinct PurchaseStatus from table').show(10)

+--------------+
|PurchaseStatus|
+--------------+
|          TRUE|
|         FALSE|
+--------------+



In [14]:
df = df.drop("ProductID","ProductName","ProductAsins","ProductKeys","ProductManufacturer")

In [53]:
from pyspark.sql.functions import count
df.groupBy('ReviewUserCity').count().show()

+--------------+-----+
|ReviewUserCity|count|
+--------------+-----+
|          null|34658|
+--------------+-----+



In [54]:
df.groupBy('ReviewUserProvince').count().show()

+------------------+-----+
|ReviewUserProvince|count|
+------------------+-----+
|              null|34658|
+------------------+-----+



In [15]:
df = df.drop("ReviewUserCity","ReviewUserProvince")

In [16]:
df = df.drop("ProductCategory","ReviewUserName","keys","ReviewID")

In [61]:
df.printSchema()

root
 |-- ProductBrand: string (nullable = true)
 |-- ProductLauncheddate: string (nullable = true)
 |-- ReviewsdateAdded: string (nullable = true)
 |-- ReviewsDateSeen: string (nullable = true)
 |-- PurchaseStatus: integer (nullable = false)
 |-- RecommendStatus: integer (nullable = false)
 |-- ReviewHelpfulStatus: string (nullable = true)
 |-- ReviewRating: string (nullable = true)
 |-- ReviewText: string (nullable = true)
 |-- ReviewTitle: string (nullable = true)



In [66]:
sqlContext.sql('select ProductBrand, Count(*) from table group by ProductBrand').show(10)

+--------------------+--------+
|        ProductBrand|count(1)|
+--------------------+--------+
|Amazon Digital Se...|      10|
|      Amazon Fire Tv|    5056|
|         Amazon Echo|     636|
|       Amazon Coco T|       1|
|         Amazon Fire|     256|
|              Amazon|   28699|
+--------------------+--------+



In [68]:
sqlContext.sql('select ReviewRating,count(*) from table group by ReviewRating order by 2 desc').show(10)

+------------+--------+
|ReviewRating|count(1)|
+------------+--------+
|           5|   23775|
|           4|    8540|
|           3|    1498|
|           1|     410|
|           2|     402|
|           0|      33|
+------------+--------+



In [47]:
df1 = sqlContext.read.csv("ProcessedDataForML.csv", header=True)

In [48]:
from pyspark.sql.functions import when
df1 = df1.withColumn("ReviewRatingClass",when(df1.ReviewRating > 4,1).otherwise(0))

In [49]:
df1 = df1.withColumn("ProductBrandID",when(df1.ProductBrand == "Amazon",1).when(df1.ProductBrand == "Amazon Fire",2).when(df1.ProductBrand == "Amazon Echo",3).when(df1.ProductBrand == "Amazon Coco T",4).when(df1.ProductBrand == "Amazon Fire Tv",5).otherwise(6) )

In [50]:
df1.registerTempTable('DataTable')

In [14]:
sqlContext.sql('select ReviewRatingClass,count(*) from DataTable group by ReviewRatingClass order by 2 desc').show(10)

+-----------------+--------+
|ReviewRatingClass|count(1)|
+-----------------+--------+
|                1|   23775|
|                0|   10883|
+-----------------+--------+



In [51]:
sqlContext.sql('select distinct ReviewHelpfulStatus from DataTable where ReviewHelpfulStatus is null').show(10)

+-------------------+
|ReviewHelpfulStatus|
+-------------------+
+-------------------+



In [15]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

In [29]:
X = df1.toPandas()['ReviewRatingClass'].values.tolist()

In [52]:
df2 = df1.drop('ProductBrand', 'ProductLauncheddate', 'ReviewsdateAdded', 'ReviewsDateSeen','ReviewRating', 'ReviewText', 'ReviewTitle', 'ReiewdAfter', 'ReiewdAfterAge', 'ReviewSeenAfter', 'ReviewSeenAfterAge')

In [53]:
df2.printSchema()

root
 |-- PurchaseStatus: string (nullable = true)
 |-- RecommendStatus: string (nullable = true)
 |-- ReviewHelpfulStatus: string (nullable = true)
 |-- ReviewRatingClass: integer (nullable = false)
 |-- ProductBrandID: integer (nullable = false)



In [54]:
(train_set, val_set, test_set) = df2.randomSplit([0.98, 0.01, 0.01], seed = 2000)

In [55]:
from pyspark.ml.feature import StringIndexer,OneHotEncoder, OneHotEncoderModel
from pyspark.ml import Pipeline

In [56]:
label_stringIdx = StringIndexer(inputCol = "ReviewRatingClass", outputCol = "label")

In [57]:
OneHot_encoder = OneHotEncoder(inputCol = "ProductBrandID", outputCol = "Brand")

In [58]:
label_stringIdx1 = StringIndexer(inputCol = "RecommendStatus", outputCol = "RecommendStatusOut")

In [59]:
label_stringIdx2 = StringIndexer(inputCol = "PurchaseStatus", outputCol = "PurchaseStatusOut")

In [60]:
label_stringIdx3 = StringIndexer(inputCol = "ReviewHelpfulStatus", outputCol = "HelpfulStatus")

In [61]:
pipeline = Pipeline(stages=[OneHot_encoder,label_stringIdx1,label_stringIdx2,label_stringIdx3,label_stringIdx])

In [62]:
pipelineFit = pipeline.fit(train_set)
train_df = pipelineFit.transform(train_set)
val_df = pipelineFit.transform(val_set)
train_df.show(5)

+--------------+---------------+-------------------+-----------------+--------------+-------------+------------------+-----------------+-------------+-----+
|PurchaseStatus|RecommendStatus|ReviewHelpfulStatus|ReviewRatingClass|ProductBrandID|        Brand|RecommendStatusOut|PurchaseStatusOut|HelpfulStatus|label|
+--------------+---------------+-------------------+-----------------+--------------+-------------+------------------+-----------------+-------------+-----+
|             0|              0|                  0|                0|             1|(6,[1],[1.0])|               1.0|              1.0|          0.0|  1.0|
|             0|              0|                  0|                0|             1|(6,[1],[1.0])|               1.0|              1.0|          0.0|  1.0|
|             0|              0|                  0|                0|             1|(6,[1],[1.0])|               1.0|              1.0|          0.0|  1.0|
|             0|              0|                  0|      

In [65]:
#from pyspark.ml.classification import LogisticRegression
#lr = LogisticRegression(maxIter=100)
#lrModel = lr.fit(train_df)
#predictions = lrModel.transform(val_df)
df2.printSchema()

root
 |-- PurchaseStatus: string (nullable = true)
 |-- RecommendStatus: string (nullable = true)
 |-- ReviewHelpfulStatus: string (nullable = true)
 |-- ReviewRatingClass: integer (nullable = false)
 |-- ProductBrandID: integer (nullable = false)



In [80]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder
CATE_FEATURES = ['PurchaseStatus', 'RecommendStatus', 'ReviewHelpfulStatus', 'ProductBrandID']
stages = [] # stages in our Pipeline
for categoricalCol in CATE_FEATURES:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()],
                                     outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

In [81]:
label_stringIdx =  StringIndexer(inputCol="ReviewRatingClass", outputCol = "label")
stages += [label_stringIdx]

In [82]:
CONTI_FEATURES  = ['ProductBrandID']
assemblerInputs = [c + "classVec" for c in CATE_FEATURES] + CONTI_FEATURES

In [83]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [84]:
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(df2)
model = pipelineModel.transform(df2)

In [85]:
model.take(1)

[Row(PurchaseStatus='1', RecommendStatus='1', ReviewHelpfulStatus='0', ReviewRatingClass=1, ProductBrandID=1, PurchaseStatusIndex=0.0, PurchaseStatusclassVec=SparseVector(1, {0: 1.0}), RecommendStatusIndex=0.0, RecommendStatusclassVec=SparseVector(1, {0: 1.0}), ReviewHelpfulStatusIndex=0.0, ReviewHelpfulStatusclassVec=SparseVector(95, {0: 1.0}), ProductBrandIDIndex=0.0, ProductBrandIDclassVec=SparseVector(5, {0: 1.0}), label=0.0, features=SparseVector(103, {0: 1.0, 1: 1.0, 2: 1.0, 97: 1.0, 102: 1.0}))]

In [86]:
from pyspark.ml.linalg import DenseVector
input_data = model.rdd.map(lambda x: (x["label"], DenseVector(x["features"])))

In [87]:
df_train = sqlContext.createDataFrame(input_data, ["label", "features"])

In [88]:
df_train.show(2)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[1.0,1.0,1.0,0.0,...|
|  0.0|[1.0,1.0,1.0,0.0,...|
+-----+--------------------+
only showing top 2 rows



In [89]:
train_data, test_data = df_train.randomSplit([.8,.2],seed=1234)

In [90]:
train_data.groupby('label').agg({'label': 'count'}).show()

+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|       18996|
|  1.0|        8695|
+-----+------------+



In [91]:
test_data.groupby('label').agg({'label': 'count'}).show()

+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|        4779|
|  1.0|        2188|
+-----+------------+



In [92]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="label",
                        featuresCol="features",
                        maxIter=10,
                        regParam=0.3)
linearModel = lr.fit(train_data)

In [93]:
predictions = linearModel.transform(test_data)

In [94]:
selected = predictions.select("label", "prediction", "probability")
selected.show(20)

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|  0.0|       1.0|[0.36306311590078...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
|  0.0|       0.0|[0.51074544785377...|
+-----+----------+--------------------+
only showing top 20 rows



In [95]:
cm = predictions.select("label", "prediction")

In [96]:
cm.filter(cm.label == cm.prediction).count() / cm.count()

0.7248457011626238